<a href="https://colab.research.google.com/github/Kinds-of-Intelligence-CFI/comparative-object-permanence/blob/multivariate-measurement-layout/analysis/measurement-layouts/object_permanence_measurement_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparative Object Permanence Measurement Layouts

Authors: K. Voudouris, J. Burden, J. Hernández-Orallo, M. Tešić

## INIT

In [ ]:
!pip install pymc --quiet
!pip install numpy --quiet
!pip install arviz --quiet
!pip install erroranalysis --quiet

In [ ]:
import arviz as az
import erroranalysis as ea
import gc
import graphviz
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pymc as pm
import random as rm
import seaborn as sns

from IPython.display import Image
from scipy import stats
from sklearn.model_selection import train_test_split
from google.colab import files
from pymc import model

print(f"Running on PyMC v{pm.__version__}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load Data


In [ ]:
agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/multivariate-measurement-layout/analysis/measurement-layouts/results_final_clean_agents_wide.csv'
agent_data = pd.read_csv(agents_url)

children_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/multivariate-measurement-layout/analysis/measurement-layouts/results_final_clean_children_wide.csv'
children_data = pd.read_csv(children_url)

synthetic_agents_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/multivariate-measurement-layout/analysis/measurement-layouts/results_synthetic_agents_wide.csv'
synthetic_agents_data = pd.read_csv(synthetic_agents_url)

long_data_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/comparative-object-permanence/multivariate-measurement-layout/analysis/results_final_clean_long.csv'
long_data = pd.read_csv(long_data_url)

## Define Some Useful Functions

In [ ]:
def logistic(x, L=1, k=1, x_0=0):
  return (L / (1 + np.exp(-k * (x-x_0))))

def logistic999(x, min, max):    # This logistic function ensures that if x is at -(max-min), we get prob 0.001, and if x is at (max-min), we get prob 0.999
  x = x - min
  max = max - min
  x = 6.90675478 * x / max
  return 1 / (1 + np.exp(-x))

def visualAcuityLOMargin(ability, goalSize, goalDistance): # must return a value between -inf and inf  (more precisely between -maxVisualAcuityAbility and maxVisualAcuityAbility)
  margin = np.log(ability)-np.log((goalDistance/goalSize))

  return margin   # Goes between -inf to inf, with logodds=0 meaning this would lead to 0.5 chance of success

def SimplePrMargin(ability, binaryFeature): # must return a value between 0 and 1
  return 1-((1-ability)*binaryFeature)  # If binaryFeature is 0 then the margin represents p(success)=1. If binaryFeature = 1 then p(success)=ability

def flatNavMargin(ability, distanceToGoal, numTurns):
  return ability - ((distanceToGoal * numTurns))

def scaledBeta(name, a, b, min, max, simpleGraph = True):
  if (simpleGraph):
    beta = pm.Beta(name, a, b)
    return beta * (max - min) + min
  else:
    beta = pm.Beta(f"{name}_raw", a, b)
    return pm.Deterministic(name, beta * (max - min) + min)


In [ ]:
# A function for pulling out the means and standard deviations for abilities of interest
def analyzeAgentResults(agentData, abilitiesToShow):

  abilityMeans = {}
  abilitySDs = {}

  for a in abilitiesToShow: #iterate through each ability, add posterior mean to dataframe, and plot posterior

    posteriorMean = float(np.mean(agentData['posterior'][a])) # calculate posterior mean
    posteriorSD = float(np.std(agentData['posterior'][a])) #calculate posterior sd
    abilityMeans[a] = posteriorMean
    abilitySDs[a] = posteriorSD

  return abilityMeans, abilitySDs

# A function for padding the testing data with 0s for prediction
def pad(testingData, trainingDataSize):
    return testingData.append(pd.Series(np.zeros(trainingDataSize-len(testingData), dtype=int)))

# A function for making predictions based on a fitted measurement layout
def predict(m, agentData, dfTest, agent_success, agent_choice, len_training, multi_var = True):
  with m:
    # set the data for prediction
    pm.set_data({"lavaPresence": pad(dfTest["lavaPresence"], len_training)})
    pm.set_data({"numChoices": pad(dfTest["numChoices"], len_training)})
    pm.set_data({"minTurnsToGoal": pad(dfTest["minNumTurnsGoal"], len_training)})
    pm.set_data({"goalDistance": pad(dfTest["minDistToGoal"], len_training)})
    if multi_var:
      pm.set_data({"minTurnsToCorrectChoice": pad(dfTest["minNumTurnsChoice"], len_training)})
      pm.set_data({"choiceDistance": pad(dfTest["minDistToCorrectChoice"], len_training)})
    pm.set_data({"goalSize": pad(dfTest["mainGoalSize"], len_training)})
    pm.set_data({"goalRight": pad(dfTest["goalRightRelToStart"], len_training)})
    pm.set_data({"goalAhead": pad(dfTest["goalCentreRelToStart"], len_training)})
    pm.set_data({"goalLeft": pad(dfTest["goalLeftRelToStart"], len_training)})
    pm.set_data({"allocentricOPTest": pad(dfTest["goalBecomesAllocentricallyOccluded"], len_training)})

    if multi_var:
      predictions = pm.sample_posterior_predictive(agentData, var_names=["choiceP", "successP"], return_inferencedata=False, predictions=True, extend_inferencedata=False)
      predictionSuccessChainRuns = predictions["successP"][:,:,0:len(dfTest)]
      predictionsSuccessInstance = np.mean(predictionSuccessChainRuns, (0,1))
      predictionChoiceChainRuns = predictions["choiceP"][:,:,0:len(dfTest)]
      predictionsChoiceInstance = np.mean(predictionChoiceChainRuns, (0,1))

      successes = dfTest[agent_success].to_numpy()
      choices = dfTest[agent_choice].to_numpy()

      return predictionsSuccessInstance, predictionsChoiceInstance, successes, choices

    else:
      predictions=pm.sample_posterior_predictive(agentData, var_names=["finalP"], return_inferencedata=False,predictions=True,extend_inferencedata=False)
      predictionChainRuns =predictions["finalP"][:,:,0:len(dfTest)]
      predictionsInstance = np.mean(predictionChainRuns, (0,1))

      return predictionsInstance,  dfTest[agent].to_numpy()


def brierScore(preds, outs):
    return 1/len(preds) * sum( (preds-outs)**2 )

def brierDecomp(preds, outs):

  brier= 1/len(preds) * sum( (preds-outs)**2 )
  ## bin predictions
  bins = np.linspace(0,1,11)
  binCenters = (bins[:-1] +bins[1:]) /2
  binPredInds = np.digitize(preds,binCenters)
  binnedPreds = bins[binPredInds]

  binTrueFreqs = np.zeros(10)
  binPredFreqs = np.zeros(10)
  binCounts = np.zeros(10)

  for i in range(10):
      idx = (preds >= bins[i]) & (preds < bins[i+1])

      binTrueFreqs[i] = np.sum(outs[idx])/np.sum(idx) if np.sum(idx) > 0 else 0
     # print(np.sum(outs[idx]), np.sum(idx), binTrueFreqs[i])
      binPredFreqs[i] = np.mean(preds[idx]) if np.sum(idx) > 0 else 0
      binCounts[i] = np.sum(idx)

  calibration = np.sum(binCounts * (binTrueFreqs - binPredFreqs) ** 2) / np.sum(binCounts) if np.sum(binCounts) > 0 else 0
  refinement = np.sum(binCounts * (binTrueFreqs *(1 - binTrueFreqs))) / np.sum(binCounts) if np.sum(binCounts) > 0 else 0
  # Compute refinement component
  #refinement = brier - calibration
  return brier, calibration, refinement



In [ ]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Define the Measurement Layout

### Univariate Modelling

Measurement layouts with just success as the dependent variable.

In [ ]:
def SetupUnivariateBasicModel(taskResultsAll, uniformAbilitySlack, agent_type, agent_name = None, sample = 500):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests.
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  if agent_type == "agent_real" or agent_type == "agent_synthetic":
    taskResults = taskResultsAll.dropna(subset = [agent_name])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults[agent_name]
  elif agent_type == "child":
    taskResults = taskResultsAll.dropna(subset = ['success'])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults['success']
  else:
    print("Agent not recognised. Quitting.")
    return



  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistance = taskResults["minDistToGoal"].max()
  maxTurns = taskResults["minNumTurnsGoal"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max() * uniformAbilitySlack
  maxPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxFlatNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).max() * uniformAbilitySlack

  # Decide 'minimum capabilities' based on easiest values in the dataset
  minDistance = taskResults["minDistToGoal"].min()
  minTurns = taskResults["minNumTurnsGoal"].min()
  minChoices = taskResults["numChoices"].min()
  minGoalSize = taskResults["mainGoalSize"].min() * (1-(uniformAbilitySlack-1))
  minPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).min()) * (1-(uniformAbilitySlack-1))
  minFlatNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).min() * (1-(uniformAbilitySlack-1))

  m = pm.Model()
  with m:

    # OP Ability
    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    # Visual acuity
    visualAcuityAbility = pm.Uniform("visualAcuityAbility", minDistance/maxGoalSize, maxDistance/minGoalSize)
    abilityMin["visualAcuityAbility"] = minDistance/maxGoalSize
    abilityMax["visualAcuityAbility"] = maxDistance/minGoalSize

    # Flat Navigation Ability
    flatNavAbility = pm.Uniform("flatNavAbility", minFlatNav, maxFlatNav)      # how much navigation is involved, i.e, how far away and how circuitous is the path to the goal?
    abilityMin["flatNavAbility"] = minFlatNav
    abilityMax["flatNavAbility"] = maxFlatNav

    # Lava Ability
    lavaAbility = scaledBeta("lavaAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 1

    # Goal Right Ability
    rightAbility = scaledBeta("rightAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 1

    # Goal Left Ability
    leftAbility = scaledBeta("leftAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 1

    # Goal Ahead Ability
    aheadAbility = scaledBeta("aheadAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 1

    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values)
    numGoals = pm.MutableData("numberOfGoals", taskResults["numGoalsAll"].values)
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values)
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values)
    goalDist = pm.MutableData("goalDistance", taskResults["minDistToGoal"].values)
    numTurns = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsGoal"])
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"])
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"])
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"])
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"].values)

    ## Margins

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility, goalSize, goalDist)))

    rightP = pm.Deterministic("rightPerformance", logistic999(SimplePrMargin(rightAbility, goalRight), min = 0, max = 1))
    aheadP = pm.Deterministic("aheadPerformance", logistic999(SimplePrMargin(aheadAbility, goalAhead), min = 0, max = 1))
    leftP = pm.Deterministic("leftPerformance", logistic999(SimplePrMargin(leftAbility, goalLeft), min = 0, max = 1))

    flatNavP = pm.Deterministic("flatNavP", logistic999(flatNavAbility - (numTurns * goalDist), min = minFlatNav, max = maxFlatNav))

    lavaP = pm.Deterministic("lavaP", logistic999(SimplePrMargin(lavaAbility, lavaPresence), min = 0, max = 1))

    navP = pm.Deterministic("navP", (flatNavP * lavaP * rightP * aheadP * leftP))

    OPPerformance = (objPermAbility  - (((goalDist*opTest) * numChoices) + (numChoices * (1-opTest))))
    objPermP = pm.Deterministic("objPermP", (logistic999(OPPerformance, min = minPermAbility, max = maxPermAbility)))

    finalP = pm.Deterministic("finalP", (objPermP * navP * visualAcuityP))

    taskSuccess = pm.Bernoulli("taskSuccess", finalP, observed=results)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = SetupUnivariateBasicModel(agent_data, uniformAbilitySlack=1, agent_type='agent_real', agent_name="success_ppo_bc_all_2023", sample = None)
gv = pm.model_graph.model_to_graphviz(m)
gv.format = "png"
gv.render(filename='viz/UnivariateNonHierarchicalMeasurementLayout.gv', directory='/content/drive/Shareddrives/comparative-object-permanence')
Image("/content/drive/Shareddrives/comparative-object-permanence/UnivariateNonHierarchicalMeasurementLayout.gv.png")
gv

In [ ]:
def SetupUnivariateHierarchicalModel(taskResults, agent_col, uniformAbilitySlack = 1):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests. Needs to be in long format
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  agent_idx, agents = taskResults[agent_col].factorize(sort=True)

  COORDS = {
    'obs': taskResults.index,
    'agent': agents ##only by type of agent, not by random seed.
    }

  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistance = taskResults["minDistToGoal"].max()
  maxTurns = taskResults["minNumTurnsGoal"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max() * uniformAbilitySlack
  maxPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxFlatNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).max() * uniformAbilitySlack

  # Decide 'minimum capabilities' based on easiest values in the dataset
  minDistance = taskResults["minDistToGoal"].min()
  minTurns = taskResults["minNumTurnsGoal"].min()
  minChoices = taskResults["numChoices"].min()
  minGoalSize = taskResults["mainGoalSize"].min() * (1-(uniformAbilitySlack-1))
  minPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).min()) * (1-(uniformAbilitySlack-1))
  minFlatNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).min() * (1-(uniformAbilitySlack-1))

  m = pm.Model(coords=COORDS)
  with m:

    #### Hyper priors ####
    V_Lava = pm.Beta('V_Lava', 1, 1)
    U_Lava = pm.HalfNormal('U_Lava',sigma = 10)
    V_Ahead = pm.Beta('V_Ahead', 1, 1)
    U_Ahead = pm.HalfNormal('U_Ahead',sigma = 10)
    V_Left = pm.Beta('V_Left', 1, 1)
    U_Left = pm.HalfNormal('U_Left',sigma = 10)
    V_Right = pm.Beta('V_Right', 1, 1)
    U_Right = pm.HalfNormal('U_Right',sigma = 10)

    V_OP = pm.Beta('V_OP', 1, 1)
    U_OP = pm.HalfNormal('U_OP',sigma = 10)
    V_VisualAcuity = pm.Beta('V_VisualAcuity', 1, 1)
    U_VisualAcuity = pm.HalfNormal('U_VisualAcuity',sigma = 10)
    V_FlatNav = pm.Beta('V_FlatNav', 1, 1)
    U_FlatNav = pm.HalfNormal('U_FlatNav',sigma = 10)

    ##ignore noise for now.

    ### Ability Priors as scaled beta distributions
    al_Lava = pm.Deterministic('al_Lava', V_Lava * U_Lava)
    be_Lava = pm.Deterministic('be_Lava', (1-V_Lava) * U_Lava)
    al_Ahead = pm.Deterministic('al_Ahead', V_Ahead * U_Ahead)
    be_Ahead = pm.Deterministic('be_Ahead', (1-V_Ahead) * U_Ahead)
    al_Left = pm.Deterministic('al_Left', V_Left * U_Left)
    be_Left = pm.Deterministic('be_Left', (1-V_Left) * U_Left)
    al_Right = pm.Deterministic('al_Right', V_Right * U_Right)
    be_Right = pm.Deterministic('be_Right', (1-V_Right) * U_Right)

    al_OP = pm.Deterministic('al_OP', V_OP * U_OP)
    be_OP = pm.Deterministic('be_OP', (1-V_OP) * U_OP)
    al_VisualAcuity = pm.Deterministic('al_VisualAcuity', V_VisualAcuity * U_VisualAcuity)
    be_VisualAcuity = pm.Deterministic('be_VisualAcuity', (1-V_VisualAcuity) * U_VisualAcuity)
    al_FlatNav = pm.Deterministic('al_FlatNav', V_FlatNav * U_FlatNav)
    be_FlatNav = pm.Deterministic('be_FlatNav', (1-V_FlatNav) * U_FlatNav)

    # Parametrise the beta distributions
    z1_Lava = pm.Gamma("z1_Lava", alpha=al_Lava, beta=1, dims = 'agent')
    z2_Lava = pm.Gamma("z2_Lava", alpha=be_Lava, beta=1, dims = 'agent')
    z1_Ahead = pm.Gamma("z1_Ahead", alpha=al_Ahead, beta=1, dims = 'agent')
    z2_Ahead = pm.Gamma("z2_Ahead", alpha=be_Ahead, beta=1, dims = 'agent')
    z1_Left = pm.Gamma("z1_Left", alpha=al_Left, beta=1, dims = 'agent')
    z2_Left = pm.Gamma("z2_Left", alpha=be_Left, beta=1, dims = 'agent')
    z1_Right = pm.Gamma("z1_Right", alpha=al_Right, beta=1, dims = 'agent')
    z2_Right = pm.Gamma("z2_Right", alpha=be_Right, beta=1, dims = 'agent')

    z1_OP = pm.Gamma("z1_OP", alpha=al_OP, beta=1, dims = 'agent')
    z2_OP = pm.Gamma("z2_OP", alpha=be_OP, beta=1, dims = 'agent')
    z1_VisualAcuity = pm.Gamma("z1_VisualAcuity", alpha=al_VisualAcuity, beta=1, dims = 'agent')
    z2_VisualAcuity = pm.Gamma("z2_VisualAcuity", alpha=be_VisualAcuity, beta=1, dims = 'agent')
    z1_FlatNav = pm.Gamma("z1_FlatNav", alpha=al_FlatNav, beta=1, dims = 'agent')
    z2_FlatNav = pm.Gamma("z2_FlatNav", alpha=be_FlatNav, beta=1, dims = 'agent')

    ## Abilities

    lavaAbility = pm.Deterministic('lavaAbility',(z1_Lava/(z1_Lava+z2_Lava)),dims='agent')
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 1

    aheadAbility = pm.Deterministic('aheadAbility',(z1_Ahead/(z1_Ahead+z2_Ahead)),dims='agent')
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 1

    leftAbility = pm.Deterministic('leftAbility',(z1_Left/(z1_Left+z2_Left)),dims='agent')
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 1

    rightAbility = pm.Deterministic('rightAbility',(z1_Right/(z1_Right+z2_Right)),dims='agent')
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 1

    objPermAbility = pm.Deterministic("objPermAbility", ((z1_OP/(z1_OP+z2_OP))*maxPermAbility)+minPermAbility, dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    abilityMin["visualAcuityAbility"] = minDistance/maxGoalSize
    abilityMax["visualAcuityAbility"] = maxDistance/minGoalSize
    visualAcuityAbility = pm.Deterministic("visualAcuityAbility", ((z1_VisualAcuity/(z1_VisualAcuity+z2_VisualAcuity))*abilityMax["visualAcuityAbility"])+abilityMin["visualAcuityAbility"], dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude

    abilityMin["flatNavAbility"] = minFlatNav
    abilityMax["flatNavAbility"] = maxFlatNav
    flatNavAbility = pm.Deterministic("flatNavAbility", ((z1_FlatNav/(z1_FlatNav+z2_FlatNav))*maxFlatNav)+minFlatNav, dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude

    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values, dims='obs')
    numGoals = pm.MutableData("numberOfGoals", taskResults["numGoalsAll"].values, dims='obs')
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values, dims='obs')
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values, dims='obs')
    goalDist = pm.MutableData("goalDistance", taskResults["minDistToGoal"].values, dims='obs')
    numTurns = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsGoal"], dims='obs')
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"], dims='obs')
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"], dims='obs')
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"], dims='obs')
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"].values, dims='obs')

    ## Margins

    rightP = pm.Deterministic("rightPerformance", logistic999(SimplePrMargin(rightAbility[agent_idx], goalRight), min = 0, max = 1),dims='obs')
    aheadP = pm.Deterministic("aheadPerformance", logistic999(SimplePrMargin(aheadAbility[agent_idx], goalAhead), min = 0, max = 1),dims='obs')
    leftP = pm.Deterministic("leftPerformance", logistic999(SimplePrMargin(leftAbility[agent_idx], goalLeft), min = 0, max = 1),dims='obs')

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility[agent_idx], goalSize, goalDist)),dims='obs')

    flatNavP = pm.Deterministic("flatNavP", logistic999(flatNavAbility[agent_idx] - (numTurns * goalDist), min = minFlatNav, max = maxFlatNav),dims='obs')

    lavaP = pm.Deterministic("lavaP", logistic999(SimplePrMargin(lavaAbility[agent_idx], lavaPresence), min = 0, max = 1),dims='obs')

    navP = pm.Deterministic("navP", (flatNavP * lavaP * rightP * aheadP * leftP),dims='obs')

    OPPerformance = pm.Deterministic("OPPerf", objPermAbility[agent_idx]  - (((goalDist*opTest) * numChoices) + (numChoices * (1-opTest))), dims='obs')

    objPermP = pm.Deterministic("objPermP", (logistic999(OPPerformance, min = minPermAbility, max = maxPermAbility)),dims='obs')

    finalP = pm.Deterministic("finalP", (objPermP * navP * visualAcuityP),dims='obs')

    taskSuccess = pm.Bernoulli("taskSuccess", finalP, observed=taskResults['success'].values,dims='obs')

  return m, abilityMin, abilityMax

In [ ]:
randomWalkers = long_data[(long_data['agent_tag_seed'].str.contains("Random"))]
hierarchical_model, abilityMins, abilityMaxs = SetupUnivariateHierarchicalModel(randomWalkers, agent_col = 'agent_tag_seed')
pm.model_to_graphviz(hierarchical_model)
gv.format = "png"
gv.render(filename='viz/UnivariateHierarchicalMeasurementLayout.gv', directory='/content/drive/Shareddrives/comparative-object-permanence')
Image("/content/drive/Shareddrives/comparative-object-permanence/UnivariateHierarchicalMeasurementLayout.gv.png")
gv

### Multivariate Modelling

Measurement layouts with success and task choice as dependent variable, in an effort to stabilise outputs for object permanence and navigation.

In [ ]:
def SetupMultivariateBasicModel(taskResultsAll, uniformAbilitySlack, agent_type, agent_name_success = None, agent_name_choice = None, sample = 500):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests.
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."

  if agent_type == "agent_real" or agent_type == "agent_synthetic":
    taskResults = taskResultsAll.dropna(subset = [agent_name_success, agent_name_choice])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults[agent_name_success]
    choices = taskResults[agent_name_choice]
  elif agent_type == "child":
    taskResults = taskResultsAll.dropna(subset = ['success', 'correctChoice'])
    if sample is not None:
      taskResults = taskResults.sample(n=sample)
    results = taskResults['success']
    choices = taskResults['correctChoice']
  else:
    print("Agent not recognised. Quitting.")
    return

  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistanceGoal = taskResults["minDistToGoal"].max()
  maxTurnsGoal = taskResults["minNumTurnsGoal"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max() * uniformAbilitySlack
  maxPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxDistanceChoice = taskResults["minDistToCorrectChoice"].max()
  maxTurnsChoice = taskResults["minNumTurnsChoice"].max()

  maxChoiceNav = ((taskResults["minDistToCorrectChoice"]*taskResults["minNumTurnsChoice"])).max() * uniformAbilitySlack # instrumental, not used in measurement layout explicitly
  maxSuccessNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).max() * uniformAbilitySlack # instrumental, not used in measurement layout explicitly

  maxFlatNav = max([maxChoiceNav, maxSuccessNav]) # pick the largest of the two products


  # Decide 'minimum capabilities' based on easiest values in the dataset
  minDistanceGoal = taskResults["minDistToGoal"].min()
  minTurnsGoal = taskResults["minNumTurnsGoal"].min()
  minChoices = taskResults["numChoices"].min()
  minGoalSize = taskResults["mainGoalSize"].min() * (1-(uniformAbilitySlack-1))
  minPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).min()) * (1-(uniformAbilitySlack-1))
  minDistanceChoice = taskResults["minDistToCorrectChoice"].min()
  minTurnsChoice = taskResults["minNumTurnsChoice"].min()

  minChoiceNav = ((taskResults["minDistToCorrectChoice"]*taskResults["minNumTurnsChoice"])).min() * (1-(uniformAbilitySlack-1)) # instrumental, not used in measurement layout explicitly
  minSuccessNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).min() * (1-(uniformAbilitySlack-1)) # instrumental, not used in measurement layout explicitly

  minFlatNav = min([minChoiceNav, minSuccessNav])

  m = pm.Model()
  with m:

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    #Visual acuity
    visualAcuityAbility = pm.Uniform("visualAcuityAbility", minDistanceGoal/maxGoalSize, maxDistanceGoal/minGoalSize)
    abilityMin["visualAcuityAbility"] = minDistanceGoal/maxGoalSize
    abilityMax["visualAcuityAbility"] = maxDistanceGoal/minGoalSize

    # Flat Navigation Ability
    flatNavAbility = pm.Uniform("flatNavAbility", minFlatNav, maxFlatNav)
    abilityMin["flatNavAbility"] = minFlatNav
    abilityMax["flatNavAbility"] = maxFlatNav

    # Lava Ability
    lavaAbility = scaledBeta("lavaAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 1

    # Goal Right Ability
    rightAbility = scaledBeta("rightAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 1

    # Goal Left Ability
    leftAbility = scaledBeta("leftAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 1

    # Goal Ahead Ability
    aheadAbility = scaledBeta("aheadAbility", 1,1, 0, 1, simpleGraph=False)
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 1

    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values)
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values)
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values)
    goalDist = pm.MutableData("goalDistance", taskResults["minDistToGoal"].values)
    numTurnsGoal = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsGoal"])
    choiceDist = pm.MutableData("choiceDistance", taskResults["minDistToCorrectChoice"].values)
    numTurnsChoice = pm.MutableData("minTurnsToCorrectChoice", taskResults["minNumTurnsChoice"].values)
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"])
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"])
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"])
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"].values)

    ## Margins

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility, goalSize, goalDist)))

    rightP = pm.Deterministic("rightPerformance", logistic999(SimplePrMargin(rightAbility, goalRight), min = 0, max = 1))
    aheadP = pm.Deterministic("aheadPerformance", logistic999(SimplePrMargin(aheadAbility, goalAhead), min = 0, max = 1))
    leftP = pm.Deterministic("leftPerformance", logistic999(SimplePrMargin(leftAbility, goalLeft), min = 0, max = 1))

    flatNavSuccessP = pm.Deterministic("flatNavSuccessP", logistic999(flatNavAbility - (numTurnsGoal * goalDist), min = minFlatNav, max = maxFlatNav))
    flatNavChoiceP = pm.Deterministic("flatNavChoiceP", logistic999(flatNavAbility - (numTurnsChoice * choiceDist), min = minFlatNav, max = maxFlatNav))

    lavaP = pm.Deterministic("lavaP", logistic999(SimplePrMargin(lavaAbility, lavaPresence), min = 0, max = 1))

    navSuccessP = pm.Deterministic("navSuccessP", (lavaP * flatNavSuccessP))
    navChoiceP = pm.Deterministic("navChoiceP", (flatNavChoiceP * aheadP * leftP * aheadP))

    OPPerformance = (objPermAbility  - (((goalDist*opTest) * numChoices) + (numChoices * (1-opTest))))
    objPermP = pm.Deterministic("objPermP", (logistic999(OPPerformance, min = minPermAbility, max = maxPermAbility)))

    choiceP = pm.Deterministic("choiceP", (navChoiceP * objPermP))
    successP = pm.Deterministic("successP", (visualAcuityP * navSuccessP * objPermP))

    taskSuccess = pm.Bernoulli("taskSuccess", successP, observed=results)
    taskChoice = pm.Bernoulli("taskChoice", choiceP, observed=choices)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = SetupMultivariateBasicModel(agent_data, uniformAbilitySlack=1, agent_type='agent_real', agent_name_success ="success_ppo_bc_all_2023", agent_name_choice = "correctChoice_ppo_bc_all_2023", sample = None)
gv = pm.model_graph.model_to_graphviz(m)
gv.format = "png"
gv.render(filename='viz/MultivariateNonHierarchicalMeasurementLayout.gv', directory='/content/drive/Shareddrives/comparative-object-permanence')
Image("/content/drive/Shareddrives/comparative-object-permanence/MultivariateNonHierarchicalMeasurementLayout.gv.png")
gv

In [ ]:
def SetupMultivariateHierarchicalModel(taskResults, uniformAbilitySlack = 1):
  """
  taskResults is the conjunction of the metadata with the successes of the agents on that set of tests. Needs to be in long format for hierarchical model
  """
  assert uniformAbilitySlack >=1, "Slack must be greater than or equal to 1."


  agent_idx, agents = taskResults['agent_tag'].factorize(sort=True)

  COORDS = {
    'obs': taskResults.index,
    'agent': agents ##only by type of agent, not by random seed.
    }

  abilityMin = {} #Initialize ability Min and max dataframes for plotting ranges
  abilityMax = {}

  #Decide "maximum capabilities" based on the hardest values in the dataset
  maxDistanceGoal = taskResults["minDistToGoal"].max()
  maxTurnsGoal = taskResults["minNumTurnsGoal"].max()
  maxChoices = taskResults["numChoices"].max()
  maxGoalSize = taskResults["mainGoalSize"].max() * uniformAbilitySlack
  maxPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).max()) * uniformAbilitySlack
  maxDistanceChoice = taskResults["minDistToCorrectChoice"].max()
  maxTurnsChoice = taskResults["minNumTurnsChoice"].max()

  maxChoiceNav = ((taskResults["minDistToCorrectChoice"]*taskResults["minNumTurnsChoice"])).max() * uniformAbilitySlack # instrumental, not used in measurement layout explicitly
  maxSuccessNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).max() * uniformAbilitySlack # instrumental, not used in measurement layout explicitly

  maxFlatNav = max([maxChoiceNav, maxSuccessNav]) # pick the largest of the two products


  # Decide 'minimum capabilities' based on easiest values in the dataset
  minDistanceGoal = taskResults["minDistToGoal"].min()
  minTurnsGoal = taskResults["minNumTurnsGoal"].min()
  minChoices = taskResults["numChoices"].min()
  minGoalSize = taskResults["mainGoalSize"].min() * (1-(uniformAbilitySlack-1))
  minPermAbility = ((taskResults["minDistToGoal"] * taskResults["numChoices"]).min()) * (1-(uniformAbilitySlack-1))
  minDistanceChoice = taskResults["minDistToCorrectChoice"].min()
  minTurnsChoice = taskResults["minNumTurnsChoice"].min()

  minChoiceNav = ((taskResults["minDistToCorrectChoice"]*taskResults["minNumTurnsChoice"])).min() * (1-(uniformAbilitySlack-1)) # instrumental, not used in measurement layout explicitly
  minSuccessNav = ((taskResults["minDistToGoal"]*taskResults["minNumTurnsGoal"])).min() * (1-(uniformAbilitySlack-1)) # instrumental, not used in measurement layout explicitly

  minFlatNav = min([minChoiceNav, minSuccessNav])

  m = pm.Model(coords=COORDS)
  with m:

    #### Hyper priors ####
    V_Lava = pm.Beta('V_Lava', 1, 1)
    U_Lava = pm.HalfNormal('U_Lava',sigma = 10)
    V_Ahead = pm.Beta('V_Ahead', 1, 1)
    U_Ahead = pm.HalfNormal('U_Ahead',sigma = 10)
    V_Left = pm.Beta('V_Left', 1, 1)
    U_Left = pm.HalfNormal('U_Left',sigma = 10)
    V_Right = pm.Beta('V_Right', 1, 1)
    U_Right = pm.HalfNormal('U_Right',sigma = 10)

    V_OP = pm.Beta('V_OP', 1, 1)
    U_OP = pm.HalfNormal('U_OP',sigma = 10)
    V_VisualAcuity = pm.Beta('V_VisualAcuity', 1, 1)
    U_VisualAcuity = pm.HalfNormal('U_VisualAcuity',sigma = 10)
    V_FlatNav = pm.Beta('V_FlatNav', 1, 1)
    U_FlatNav = pm.HalfNormal('U_FlatNav',sigma = 10)

    ##ignore noise for now.

    ### Ability Priors as scaled beta distributions
    al_Lava = pm.Deterministic('al_Lava', V_Lava * U_Lava)
    be_Lava = pm.Deterministic('be_Lava', (1-V_Lava) * U_Lava)
    al_Ahead = pm.Deterministic('al_Ahead', V_Ahead * U_Ahead)
    be_Ahead = pm.Deterministic('be_Ahead', (1-V_Ahead) * U_Ahead)
    al_Left = pm.Deterministic('al_Left', V_Left * U_Left)
    be_Left = pm.Deterministic('be_Left', (1-V_Left) * U_Left)
    al_Right = pm.Deterministic('al_Right', V_Right * U_Right)
    be_Right = pm.Deterministic('be_Right', (1-V_Right) * U_Right)

    al_OP = pm.Deterministic('al_OP', V_OP * U_OP)
    be_OP = pm.Deterministic('be_OP', (1-V_OP) * U_OP)
    al_VisualAcuity = pm.Deterministic('al_VisualAcuity', V_VisualAcuity * U_VisualAcuity)
    be_VisualAcuity = pm.Deterministic('be_VisualAcuity', (1-V_VisualAcuity) * U_VisualAcuity)
    al_FlatNav = pm.Deterministic('al_FlatNav', V_FlatNav * U_FlatNav)
    be_FlatNav = pm.Deterministic('be_FlatNav', (1-V_FlatNav) * U_FlatNav)

    # Parametrise the beta distributions
    z1_Lava = pm.Gamma("z1_Lava", alpha=al_Lava, beta=1, dims = 'agent')
    z2_Lava = pm.Gamma("z2_Lava", alpha=be_Lava, beta=1, dims = 'agent')
    z1_Ahead = pm.Gamma("z1_Ahead", alpha=al_Ahead, beta=1, dims = 'agent')
    z2_Ahead = pm.Gamma("z2_Ahead", alpha=be_Ahead, beta=1, dims = 'agent')
    z1_Left = pm.Gamma("z1_Left", alpha=al_Left, beta=1, dims = 'agent')
    z2_Left = pm.Gamma("z2_Left", alpha=be_Left, beta=1, dims = 'agent')
    z1_Right = pm.Gamma("z1_Right", alpha=al_Right, beta=1, dims = 'agent')
    z2_Right = pm.Gamma("z2_Right", alpha=be_Right, beta=1, dims = 'agent')

    z1_OP = pm.Gamma("z1_OP", alpha=al_OP, beta=1, dims = 'agent')
    z2_OP = pm.Gamma("z2_OP", alpha=be_OP, beta=1, dims = 'agent')
    z1_VisualAcuity = pm.Gamma("z1_VisualAcuity", alpha=al_VisualAcuity, beta=1, dims = 'agent')
    z2_VisualAcuity = pm.Gamma("z2_VisualAcuity", alpha=be_VisualAcuity, beta=1, dims = 'agent')
    z1_FlatNav = pm.Gamma("z1_FlatNav", alpha=al_FlatNav, beta=1, dims = 'agent')
    z2_FlatNav = pm.Gamma("z2_FlatNav", alpha=be_FlatNav, beta=1, dims = 'agent')

    ## Abilities

    lavaAbility = pm.Deterministic('lavaAbility',(z1_Lava/(z1_Lava+z2_Lava)),dims='agent')
    abilityMin["lavaAbility"] = 0
    abilityMax["lavaAbility"] = 1

    aheadAbility = pm.Deterministic('aheadAbility',(z1_Ahead/(z1_Ahead+z2_Ahead)),dims='agent')
    abilityMin["aheadAbility"] = 0
    abilityMax["aheadAbility"] = 1

    leftAbility = pm.Deterministic('leftAbility',(z1_Left/(z1_Left+z2_Left)),dims='agent')
    abilityMin["leftAbility"] = 0
    abilityMax["leftAbility"] = 1

    rightAbility = pm.Deterministic('rightAbility',(z1_Right/(z1_Right+z2_Right)),dims='agent')
    abilityMin["rightAbility"] = 0
    abilityMax["rightAbility"] = 1

    objPermAbility = pm.Deterministic("objPermAbility", ((z1_OP/(z1_OP+z2_OP))*maxPermAbility)+minPermAbility, dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude
    abilityMin["objPermAbility"] = minPermAbility
    abilityMax["objPermAbility"] = maxPermAbility

    abilityMin["visualAcuityAbility"] = minDistanceGoal/maxGoalSize
    abilityMax["visualAcuityAbility"] = minDistanceGoal/minGoalSize
    visualAcuityAbility = pm.Deterministic("visualAcuityAbility", ((z1_VisualAcuity/(z1_VisualAcuity+z2_VisualAcuity))*abilityMax["visualAcuityAbility"])+abilityMin["visualAcuityAbility"], dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude

    abilityMin["flatNavAbility"] = minFlatNav
    abilityMax["flatNavAbility"] = maxFlatNav
    flatNavAbility = pm.Deterministic("flatNavAbility", ((z1_FlatNav/(z1_FlatNav+z2_FlatNav))*maxFlatNav)+minFlatNav, dims='agent')  # [0,maxPermAbility] This is the same as above, but we have the ability in the right magnitude

    ## Environment variables as Deterministic (about the instance)

    lavaPresence = pm.MutableData("lavaPresence", taskResults["lavaPresence"].values, dims='obs')
    numChoices = pm.MutableData("numChoices", taskResults["numChoices"].values, dims='obs')
    goalSize = pm.MutableData("goalSize",taskResults["mainGoalSize"].values, dims='obs')
    goalDist = pm.MutableData("goalDistance", taskResults["minDistToGoal"].values, dims='obs')
    numTurnsGoal = pm.MutableData("minTurnsToGoal", taskResults["minNumTurnsGoal"], dims='obs')
    choiceDist = pm.MutableData("choiceDistance", taskResults["minDistToCorrectChoice"].values, dims='obs')
    numTurnsChoice = pm.MutableData("minTurnsToCorrectChoice", taskResults["minNumTurnsChoice"].values, dims='obs')
    goalRight = pm.MutableData("goalRight", taskResults["goalRightRelToStart"], dims='obs')
    goalAhead = pm.MutableData("goalAhead", taskResults["goalCentreRelToStart"], dims='obs')
    goalLeft = pm.MutableData("goalLeft", taskResults["goalLeftRelToStart"], dims='obs')
    opTest = pm.MutableData("allocentricOPTest", taskResults["goalBecomesAllocentricallyOccluded"].values, dims='obs')

    ## Margins

    rightP = pm.Deterministic("rightPerformance", logistic999(SimplePrMargin(rightAbility[agent_idx], goalRight), min = 0, max = 1),dims='obs')
    aheadP = pm.Deterministic("aheadPerformance", logistic999(SimplePrMargin(aheadAbility[agent_idx], goalAhead), min = 0, max = 1),dims='obs')
    leftP = pm.Deterministic("leftPerformance", logistic999(SimplePrMargin(leftAbility[agent_idx], goalLeft), min = 0, max = 1),dims='obs')

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic(visualAcuityLOMargin(visualAcuityAbility[agent_idx], goalSize, goalDist)),dims='obs')

    flatNavSuccessP = pm.Deterministic("flatNavSuccessP", logistic999(flatNavAbility[agent_idx] - (numTurnsGoal * goalDist), min = minFlatNav, max = maxFlatNav),dims='obs')
    flatNavChoiceP = pm.Deterministic("flatNavChoiceP", logistic999(flatNavAbility[agent_idx] - (numTurnsChoice * choiceDist), min = minFlatNav, max = maxFlatNav),dims='obs')

    lavaP = pm.Deterministic("lavaP", logistic999(SimplePrMargin(lavaAbility[agent_idx], lavaPresence), min = 0, max = 1),dims='obs')

    navSuccessP = pm.Deterministic("navSuccessP", (lavaP * flatNavSuccessP),dims='obs')
    navChoiceP = pm.Deterministic("navChoiceP", (flatNavChoiceP * aheadP * leftP * rightP),dims='obs')

    OPPerformance = pm.Deterministic("OPPerf", objPermAbility[agent_idx]  - (((goalDist*opTest) * numChoices) + (numChoices * (1-opTest))), dims='obs') #if there are multiple choices, the agent has to have some object permanence to solve the task.
    objPermP = pm.Deterministic("objPermP", (logistic999(OPPerformance, min = minPermAbility, max = maxPermAbility)),dims='obs')

    choiceP = pm.Deterministic("choiceP", (navChoiceP * objPermP),dims='obs')
    successP = pm.Deterministic("successP", (visualAcuityP * navSuccessP * objPermP),dims='obs')

    taskSuccess = pm.Bernoulli("taskSuccess", successP, observed=taskResults['success'].values,dims='obs')
    taskChoice = pm.Bernoulli("taskChoice", choiceP, observed=taskResults['correctChoice'].values,dims='obs')

  return m, abilityMin, abilityMax

In [ ]:
agent_data = long_data[(long_data['agent_type_gen'].str.contains("Child"))].dropna(subset = ['success', 'correctChoice'])

In [ ]:
m, abilityMin, abilityMax = SetupMultivariateHierarchicalModel(agent_data, uniformAbilitySlack=1)
gv = pm.model_graph.model_to_graphviz(m)
gv.format = "png"
gv.render(filename='viz/MultivariateHierarchicalMeasurementLayout.gv', directory='/content/drive/Shareddrives/comparative-object-permanence')
Image("/content/drive/Shareddrives/comparative-object-permanence/MultivariateHierarchicalMeasurementLayout.gv.png")
gv

## Run Complete Multivariate Measurement Layouts

In [ ]:
ppo_dreamer_names = ["dreamer_bc_all_9999",
                     "dreamer_bc_opc_all_9999",
                     "dreamer_bc_opc_strat_9999",
                     "dreamer_bc_opc_opt_all_9999",
                     "dreamer_bc_opc_opt_strat_9999",
                     "ppo_bc_all_2023",
                     "ppo_bc_opc_strat_2023",
                     "ppo_bc_opc_all_2023",
                     "ppo_bc_opc_opt_strat_2023",
                     "ppo_bc_opc_opt_all_2023"]


synthetic_agent_names = ["perfectAgent",
                         "failedAgent",
                         "noOPAgent",
                         "lowVisualAcuityAgent",
                         "poorNavigationOPAgent",
                         "poorLavaOPAgent"]


reference_agents_types = ["Child",
                          "Heuristic Agent",
                          "Random Agent"]


In [ ]:
synthetic_agents_training, synthetic_agents_test = train_test_split(synthetic_agents_data, test_size = 0.2)
RL_agents_training, RL_agents_test = train_test_split(agent_data, test_size=0.2)

In [ ]:
abilitiesToShow = ["objPermAbility", "visualAcuityAbility", "flatNavAbility", "lavaAbility", "rightAbility", "leftAbility", "aheadAbility"]

pymc_sample_num = 2000

slack = 1

In [ ]:
agentName = []
modelBrierScoreSuccess = []
modelBrierScoreChoice = []
aggregateBrierScoreSuccess = []
aggregateBrierScoreChoice = []
modelCalibrationSuccess = []
modelCalibrationChoice = []
aggregateCalibrationSuccess = []
aggregateCalibrationChoice = []
modelRefinementSuccess = []
modelRefinementChoice = []
aggregateRefinementSuccess = []
aggregateRefinementChoice = []
modelBetterThanAggSuccess = []
modelBetterThanAggChoice = []
meanSuccess = []
meanChoice = []

OPAbilityAllMean = []
navAbilityAllMean = []
visualAcuityAbilityAllMean = []
flatNavAbilityAllMean = []
lavaAbilityAllMean = []
rightAbilityAllMean = []
leftAbilityAllMean = []
aheadAbilityAllMean = []

OPAbilityAllSD = []
navAbilityAllSD = []
visualAcuityAbilityAllSD = []
flatNavAbilityAllSD = []
lavaAbilityAllSD = []
rightAbilityAllSD = []
leftAbilityAllSD = []
aheadAbilityAllSD = []

OPAbilityAllMin = []
navAbilityAllMin = []
visualAcuityAbilityAllMin = []
flatNavAbilityAllMin = []
lavaAbilityAllMin = []
rightAbilityAllMin = []
leftAbilityAllMin = []
aheadAbilityAllMin = []

OPAbilityAllMax = []
navAbilityAllMax = []
visualAcuityAbilityAllMax = []
flatNavAbilityAllMax = []
lavaAbilityAllMax = []
rightAbilityAllMax = []
leftAbilityAllMax = []
aheadAbilityAllMax = []

In [ ]:
training_data_model_folder = '/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/training_set_model'
full_data_model_folder = '/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/full_data_model'
figures_folder = '/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/figures'

In [ ]:
for agent in synthetic_agent_names:
  model_train, ability_min, ability_max = SetupMultivariateBasicModel(synthetic_agents_training, uniformAbilitySlack=1, agent_type='agent_synthetic', agent_name_success = f"{agent}_success", agent_name_choice = f"{agent}_choice", sample = None)
  with model_train:
    data_training = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(model_train, f"{training_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  predictionsSuccessInstance, predictionsChoiceInstance, successes, choices = predict(model_train, data_training, synthetic_agents_test, f"{agent}_success", f"{agent}_choice", len(synthetic_agents_training), multi_var=True)

  agentBrierScoreSuccess, agentCalibrationSuccess, agentRefinementSuccess = brierDecomp(predictionsSuccessInstance, successes)
  agentAggBrierScoreSuccess, agentAggCalibrationSuccess, agentAggRefinementSuccess = brierDecomp(np.repeat(np.mean(synthetic_agents_training[f"{agent}_success"]), len(successes)), successes)
  agentBrierScoreChoice, agentCalibrationChoice, agentRefinementChoice = brierDecomp(predictionsChoiceInstance, choices)
  agentAggBrierScoreChoice, agentAggCalibrationChoice, agentAggRefinementChoice = brierDecomp(np.repeat(np.mean(synthetic_agents_training[f"{agent}_choice"]), len(choices)), choices)

  aggregateBrierScoreSuccess.append(agentAggBrierScoreSuccess)
  aggregateBrierScoreChoice.append(agentAggBrierScoreChoice)
  aggregateCalibrationSuccess.append(agentAggCalibrationSuccess)
  aggregateCalibrationChoice.append(agentAggCalibrationChoice)
  aggregateRefinementSuccess.append(agentAggRefinementSuccess)
  aggregateRefinementChoice.append(agentAggRefinementChoice)

  modelBrierScoreSuccess.append(agentBrierScoreSuccess)
  modelBrierScoreChoice.append(agentBrierScoreChoice)
  modelCalibrationSuccess.append(agentCalibrationSuccess)
  modelCalibrationChoice.append(agentCalibrationChoice)
  modelRefinementSuccess.append(agentRefinementSuccess)
  modelRefinementChoice.append(agentRefinementChoice)

  if agentBrierScoreSuccess < agentAggBrierScoreSuccess:
    modelBetterThanAggSuccess.append(True)
  else:
    modelBetterThanAggSuccess.append(False)

  if agentBrierScoreChoice < agentAggBrierScoreChoice:
    modelBetterThanAggChoice.append(True)
  else:
    modelBetterThanAggChoice.append(False)

  # now train on all data

  model_all, ability_min, ability_max = SetupMultivariateBasicModel(synthetic_agents_data, uniformAbilitySlack=1, agent_type='agent_synthetic', agent_name_success = f"{agent}_success", agent_name_choice = f"{agent}_choice", sample = None)
  with model_all:
    data_all = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(data_all, f"{full_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  mu, sd  = analyzeAgentResults(data_all, abilitiesToShow)

  OPAbilityAllMean.append(mu["objPermAbility"])
  visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
  flatNavAbilityAllMean.append(mu["flatNavAbility"])
  lavaAbilityAllMean.append(mu["lavaAbility"])
  rightAbilityAllMean.append(mu["rightAbility"])
  leftAbilityAllMean.append(mu["leftAbility"])
  aheadAbilityAllMean.append(mu["aheadAbility"])

  OPAbilityAllSD.append(sd["objPermAbility"])
  visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
  flatNavAbilityAllSD.append(sd["flatNavAbility"])
  lavaAbilityAllSD.append(sd["lavaAbility"])
  rightAbilityAllSD.append(sd["rightAbility"])
  leftAbilityAllSD.append(sd["leftAbility"])
  aheadAbilityAllSD.append(sd["aheadAbility"])

  OPAbilityAllMin.append(ability_min["objPermAbility"])
  visualAcuityAbilityAllMin.append(ability_min["visualAcuityAbility"])
  flatNavAbilityAllMin.append(ability_min["flatNavAbility"])
  lavaAbilityAllMin.append(ability_min["lavaAbility"])
  rightAbilityAllMin.append(ability_min["rightAbility"])
  leftAbilityAllMin.append(ability_min["leftAbility"])
  aheadAbilityAllMin.append(ability_min["aheadAbility"])

  OPAbilityAllMax.append(ability_max["objPermAbility"])
  visualAcuityAbilityAllMax.append(ability_max["visualAcuityAbility"])
  flatNavAbilityAllMax.append(ability_max["flatNavAbility"])
  lavaAbilityAllMax.append(ability_max["lavaAbility"])
  rightAbilityAllMax.append(ability_max["rightAbility"])
  leftAbilityAllMax.append(ability_max["leftAbility"])
  aheadAbilityAllMax.append(ability_max["aheadAbility"])

  meanSuccess.append(np.mean(synthetic_agents_data[f"{agent}_success"]))
  meanChoice.append(np.mean(synthetic_agents_data[f"{agent}_choice"]))

  agentName.append(agent)

  finalDF = pd.DataFrame({"Agent Name": agentName,
                          "Model Brier Score Success":modelBrierScoreSuccess,
                          "Model Brier Score Choice":modelBrierScoreChoice,
                          "Aggregate Brier Score Success": aggregateBrierScoreSuccess,
                          "Aggregate Brier Score Choice": aggregateBrierScoreChoice,
                          "Model Calibration Success": modelCalibrationSuccess,
                          "Model Calibration Choice": modelCalibrationChoice,
                          "Aggregate Calibration Success":aggregateCalibrationSuccess,
                          "Aggregate Calibration Choice":aggregateCalibrationChoice,
                          "Model Refinement Success" : modelRefinementSuccess,
                          "Model Refinement Choice" : modelRefinementChoice,
                          "Aggregate Refinement Success" : aggregateRefinementSuccess,
                          "Aggregate Refinement Choice" : aggregateRefinementChoice,
                          "Model Better? (Based on Brier Score - Success)":modelBetterThanAggSuccess,
                          "Model Better? (Based on Brier Score - Choice)":modelBetterThanAggChoice,
                          "Average Success":meanSuccess,
                          "Average Correct Chocie":meanChoice,
                          "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                          "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                          "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                          "Lava Ability Mean (All)" : lavaAbilityAllMean,
                          "Right Ability Mean (All)" : rightAbilityAllMean,
                          "Left Ability Mean (All)" : leftAbilityAllMean,
                          "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                          "Object Permanence Ability SD (All)": OPAbilityAllSD,
                          "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                          "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                          "Lava Ability SD (All)" : lavaAbilityAllSD,
                          "Right Ability SD (All)" : rightAbilityAllSD,
                          "Left Ability SD (All)" : leftAbilityAllSD,
                          "Ahead Ability SD (All)" : aheadAbilityAllSD,
                          "Object Permanence Ability Min (All)": OPAbilityAllMin,
                          "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                          "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                          "Lava Ability Min (All)" : lavaAbilityAllMin,
                          "Right Ability Min (All)" : rightAbilityAllMin,
                          "Left Ability Min (All)" : leftAbilityAllMin,
                          "Ahead Ability Min (All)" : aheadAbilityAllMin,
                          "Object Permanence Ability Max (All)": OPAbilityAllMax,
                          "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                          "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                          "Lava Ability Max (All)" : lavaAbilityAllMax,
                          "Right Ability Max (All)" : rightAbilityAllMax,
                          "Left Ability Max (All)" : leftAbilityAllMax,
                          "Ahead Ability Max (All)" : aheadAbilityAllMax,})
  finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/AgentResults.csv", index = False)

  ## figures

  for ability in abilitiesToShow:
    trace_plot = az.plot_trace(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/traceplot_{agent}_{ability}.png")

    forest_plot = az.plot_forest(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/forest_{agent}_{ability}.png")

  energy_plot = az.plot_energy(data=data_all)
  plt.savefig(f"{figures_folder}/energyplot_{agent}.png")

  summary = az.summary(data_all['posterior'][abilitiesToShow])
  save_object(summary, f"{full_data_model_folder}/summary_{agent}.pkl")
  summary.to_csv(f"{full_data_model_folder}/summary_{agent}.csv", index = False)

  print(f"Agent: {agent}")
  print(summary)


In [ ]:
for agent in ppo_dreamer_names:
  model_train, ability_min, ability_max = SetupMultivariateBasicModel(RL_agents_training, uniformAbilitySlack=1, agent_type='agent_real', agent_name_success = f"success_{agent}", agent_name_choice = f"correctChoice_{agent}", sample = None)
  with model_train:
    data_training = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(model_train, f"{training_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  predictionsSuccessInstance, predictionsChoiceInstance, successes, choices = predict(model_train, data_training, RL_agents_test, f"success_{agent}", f"correctChoice_{agent}", len(RL_agents_training), multi_var=True)

  agentBrierScoreSuccess, agentCalibrationSuccess, agentRefinementSuccess = brierDecomp(predictionsSuccessInstance, successes)
  agentAggBrierScoreSuccess, agentAggCalibrationSuccess, agentAggRefinementSuccess = brierDecomp(np.repeat(np.mean(RL_agents_training[f"success_{agent}"]), len(successes)), successes)
  agentBrierScoreChoice, agentCalibrationChoice, agentRefinementChoice = brierDecomp(predictionsChoiceInstance, choices)
  agentAggBrierScoreChoice, agentAggCalibrationChoice, agentAggRefinementChoice = brierDecomp(np.repeat(np.mean(RL_agents_training[f"correctChoice_{agent}"]), len(choices)), choices)

  aggregateBrierScoreSuccess.append(agentAggBrierScoreSuccess)
  aggregateBrierScoreChoice.append(agentAggBrierScoreChoice)
  aggregateCalibrationSuccess.append(agentAggCalibrationSuccess)
  aggregateCalibrationChoice.append(agentAggCalibrationChoice)
  aggregateRefinementSuccess.append(agentAggRefinementSuccess)
  aggregateRefinementChoice.append(agentAggRefinementChoice)

  modelBrierScoreSuccess.append(agentBrierScoreSuccess)
  modelBrierScoreChoice.append(agentBrierScoreChoice)
  modelCalibrationSuccess.append(agentCalibrationSuccess)
  modelCalibrationChoice.append(agentCalibrationChoice)
  modelRefinementSuccess.append(agentRefinementSuccess)
  modelRefinementChoice.append(agentRefinementChoice)

  if agentBrierScoreSuccess < agentAggBrierScoreSuccess:
    modelBetterThanAggSuccess.append(True)
  else:
    modelBetterThanAggSuccess.append(False)

  if agentBrierScoreChoice < agentAggBrierScoreChoice:
    modelBetterThanAggChoice.append(True)
  else:
    modelBetterThanAggChoice.append(False)

  # now train on all data

  model_all, ability_min, ability_max = SetupMultivariateBasicModel(agent_data, uniformAbilitySlack=1, agent_type='agent_real', agent_name_success = f"success_{agent}", agent_name_choice = f"correctChoice_{agent}", sample = None)
  with model_all:
    data_all = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(data_all, f"{full_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  mu, sd  = analyzeAgentResults(data_all, abilitiesToShow)

  OPAbilityAllMean.append(mu["objPermAbility"])
  visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
  flatNavAbilityAllMean.append(mu["flatNavAbility"])
  lavaAbilityAllMean.append(mu["lavaAbility"])
  rightAbilityAllMean.append(mu["rightAbility"])
  leftAbilityAllMean.append(mu["leftAbility"])
  aheadAbilityAllMean.append(mu["aheadAbility"])

  OPAbilityAllSD.append(sd["objPermAbility"])
  visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
  flatNavAbilityAllSD.append(sd["flatNavAbility"])
  lavaAbilityAllSD.append(sd["lavaAbility"])
  rightAbilityAllSD.append(sd["rightAbility"])
  leftAbilityAllSD.append(sd["leftAbility"])
  aheadAbilityAllSD.append(sd["aheadAbility"])

  OPAbilityAllMin.append(ability_min["objPermAbility"])
  visualAcuityAbilityAllMin.append(ability_min["visualAcuityAbility"])
  flatNavAbilityAllMin.append(ability_min["flatNavAbility"])
  lavaAbilityAllMin.append(ability_min["lavaAbility"])
  rightAbilityAllMin.append(ability_min["rightAbility"])
  leftAbilityAllMin.append(ability_min["leftAbility"])
  aheadAbilityAllMin.append(ability_min["aheadAbility"])

  OPAbilityAllMax.append(ability_max["objPermAbility"])
  visualAcuityAbilityAllMax.append(ability_max["visualAcuityAbility"])
  flatNavAbilityAllMax.append(ability_max["flatNavAbility"])
  lavaAbilityAllMax.append(ability_max["lavaAbility"])
  rightAbilityAllMax.append(ability_max["rightAbility"])
  leftAbilityAllMax.append(ability_max["leftAbility"])
  aheadAbilityAllMax.append(ability_max["aheadAbility"])

  meanSuccess.append(np.mean(agent_data[f"success_{agent}"]))
  meanChoice.append(np.mean(agent_data[f"correctChoice_{agent}"]))

  agentName.append(agent)

  finalDF = pd.DataFrame({"Agent Name": agentName,
                          "Model Brier Score Success":modelBrierScoreSuccess,
                          "Model Brier Score Choice":modelBrierScoreChoice,
                          "Aggregate Brier Score Success": aggregateBrierScoreSuccess,
                          "Aggregate Brier Score Choice": aggregateBrierScoreChoice,
                          "Model Calibration Success": modelCalibrationSuccess,
                          "Model Calibration Choice": modelCalibrationChoice,
                          "Aggregate Calibration Success":aggregateCalibrationSuccess,
                          "Aggregate Calibration Choice":aggregateCalibrationChoice,
                          "Model Refinement Success" : modelRefinementSuccess,
                          "Model Refinement Choice" : modelRefinementChoice,
                          "Aggregate Refinement Success" : aggregateRefinementSuccess,
                          "Aggregate Refinement Choice" : aggregateRefinementChoice,
                          "Model Better? (Based on Brier Score - Success)":modelBetterThanAggSuccess,
                          "Model Better? (Based on Brier Score - Choice)":modelBetterThanAggChoice,
                          "Average Success":meanSuccess,
                          "Average Correct Chocie":meanChoice,
                          "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                          "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                          "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                          "Lava Ability Mean (All)" : lavaAbilityAllMean,
                          "Right Ability Mean (All)" : rightAbilityAllMean,
                          "Left Ability Mean (All)" : leftAbilityAllMean,
                          "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                          "Object Permanence Ability SD (All)": OPAbilityAllSD,
                          "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                          "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                          "Lava Ability SD (All)" : lavaAbilityAllSD,
                          "Right Ability SD (All)" : rightAbilityAllSD,
                          "Left Ability SD (All)" : leftAbilityAllSD,
                          "Ahead Ability SD (All)" : aheadAbilityAllSD,
                          "Object Permanence Ability Min (All)": OPAbilityAllMin,
                          "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                          "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                          "Lava Ability Min (All)" : lavaAbilityAllMin,
                          "Right Ability Min (All)" : rightAbilityAllMin,
                          "Left Ability Min (All)" : leftAbilityAllMin,
                          "Ahead Ability Min (All)" : aheadAbilityAllMin,
                          "Object Permanence Ability Max (All)": OPAbilityAllMax,
                          "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                          "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                          "Lava Ability Max (All)" : lavaAbilityAllMax,
                          "Right Ability Max (All)" : rightAbilityAllMax,
                          "Left Ability Max (All)" : leftAbilityAllMax,
                          "Ahead Ability Max (All)" : aheadAbilityAllMax,})
  finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/AgentResults.csv", index = False)

  ## figures

  for ability in abilitiesToShow:
    trace_plot = az.plot_trace(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/traceplot_{agent}_{ability}.png")

    forest_plot = az.plot_forest(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/forest_{agent}_{ability}.png")

  energy_plot = az.plot_energy(data=data_all)
  plt.savefig(f"{figures_folder}/energyplot_{agent}.png")

  summary = az.summary(data_all['posterior'][abilitiesToShow])
  save_object(summary, f"{full_data_model_folder}/summary_{agent}.pkl")
  summary.to_csv(f"{full_data_model_folder}/summary_{agent}.csv", index = False)

  print(f"Agent: {agent}")
  print(summary)


In [ ]:
for agent in reference_agents_types:
  agent_subset = long_data[(long_data['agent_type_gen'].str.contains(agent))].dropna(subset = ['success', 'correctChoice'])
  train_data, test_data = train_test_split(agent_subset, test_size = 0.2)

  model_train, ability_min, ability_max = SetupMultivariateHierarchicalModel(train_data, uniformAbilitySlack=1)
  with model_train:
    data_training = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(model_train, f"{training_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  predictionsSuccessInstance, predictionsChoiceInstance, successes, choices = predict(model_train, data_training, test_data, "success", "correctChoice", len(train_data), multi_var=True)

  agentBrierScoreSuccess, agentCalibrationSuccess, agentRefinementSuccess = brierDecomp(predictionsSuccessInstance, successes)
  agentAggBrierScoreSuccess, agentAggCalibrationSuccess, agentAggRefinementSuccess = brierDecomp(np.repeat(np.mean(train_data[f"{agent}_success"]), len(successes)), successes)
  agentBrierScoreChoice, agentCalibrationChoice, agentRefinementChoice = brierDecomp(predictionsChoiceInstance, choices)
  agentAggBrierScoreChoice, agentAggCalibrationChoice, agentAggRefinementChoice = brierDecomp(np.repeat(np.mean(train_data[f"{agent}_choice"]), len(choices)), choices)

  aggregateBrierScoreSuccess.append(agentAggBrierScoreSuccess)
  aggregateBrierScoreChoice.append(agentAggBrierScoreChoice)
  aggregateCalibrationSuccess.append(agentAggCalibrationSuccess)
  aggregateCalibrationChoice.append(agentAggCalibrationChoice)
  aggregateRefinementSuccess.append(agentAggRefinementSuccess)
  aggregateRefinementChoice.append(agentAggRefinementChoice)

  modelBrierScoreSuccess.append(agentBrierScoreSuccess)
  modelBrierScoreChoice.append(agentBrierScoreChoice)
  modelCalibrationSuccess.append(agentCalibrationSuccess)
  modelCalibrationChoice.append(agentCalibrationChoice)
  modelRefinementSuccess.append(agentRefinementSuccess)
  modelRefinementChoice.append(agentRefinementChoice)

  if agentBrierScoreSuccess < agentAggBrierScoreSuccess:
    modelBetterThanAggSuccess.append(True)
  else:
    modelBetterThanAggSuccess.append(False)

  if agentBrierScoreChoice < agentAggBrierScoreChoice:
    modelBetterThanAggChoice.append(True)
  else:
    modelBetterThanAggChoice.append(False)

  # now train on all data

  model_all, ability_min, ability_max = SetupMultivariateHierarchicalModel(agent_subset, uniformAbilitySlack=1)
  with model_all:
    data_all = pm.sample(pymc_sample_num, target_accept=0.95)

  #save_object(data_all, f"{full_data_model_folder}/model_{agent}.pkl") #save the model just in case we need it again.

  mu, sd  = analyzeAgentResults(data_all, abilitiesToShow)

  OPAbilityAllMean.append(mu["objPermAbility"])
  visualAcuityAbilityAllMean.append(mu["visualAcuityAbility"])
  flatNavAbilityAllMean.append(mu["flatNavAbility"])
  lavaAbilityAllMean.append(mu["lavaAbility"])
  rightAbilityAllMean.append(mu["rightAbility"])
  leftAbilityAllMean.append(mu["leftAbility"])
  aheadAbilityAllMean.append(mu["aheadAbility"])

  OPAbilityAllSD.append(sd["objPermAbility"])
  visualAcuityAbilityAllSD.append(sd["visualAcuityAbility"])
  flatNavAbilityAllSD.append(sd["flatNavAbility"])
  lavaAbilityAllSD.append(sd["lavaAbility"])
  rightAbilityAllSD.append(sd["rightAbility"])
  leftAbilityAllSD.append(sd["leftAbility"])
  aheadAbilityAllSD.append(sd["aheadAbility"])

  OPAbilityAllMin.append(ability_min["objPermAbility"])
  visualAcuityAbilityAllMin.append(ability_min["visualAcuityAbility"])
  flatNavAbilityAllMin.append(ability_min["flatNavAbility"])
  lavaAbilityAllMin.append(ability_min["lavaAbility"])
  rightAbilityAllMin.append(ability_min["rightAbility"])
  leftAbilityAllMin.append(ability_min["leftAbility"])
  aheadAbilityAllMin.append(ability_min["aheadAbility"])

  OPAbilityAllMax.append(ability_max["objPermAbility"])
  visualAcuityAbilityAllMax.append(ability_max["visualAcuityAbility"])
  flatNavAbilityAllMax.append(ability_max["flatNavAbility"])
  lavaAbilityAllMax.append(ability_max["lavaAbility"])
  rightAbilityAllMax.append(ability_max["rightAbility"])
  leftAbilityAllMax.append(ability_max["leftAbility"])
  aheadAbilityAllMax.append(ability_max["aheadAbility"])

  meanSuccess.append(np.mean(agent_subset['success']))
  meanChoice.append(np.mean(agent_subset['correctChoice']))

  agentName.append(agent)

  finalDF = pd.DataFrame({"Agent Name": agentName,
                          "Model Brier Score Success":modelBrierScoreSuccess,
                          "Model Brier Score Choice":modelBrierScoreChoice,
                          "Aggregate Brier Score Success": aggregateBrierScoreSuccess,
                          "Aggregate Brier Score Choice": aggregateBrierScoreChoice,
                          "Model Calibration Success": modelCalibrationSuccess,
                          "Model Calibration Choice": modelCalibrationChoice,
                          "Aggregate Calibration Success":aggregateCalibrationSuccess,
                          "Aggregate Calibration Choice":aggregateCalibrationChoice,
                          "Model Refinement Success" : modelRefinementSuccess,
                          "Model Refinement Choice" : modelRefinementChoice,
                          "Aggregate Refinement Success" : aggregateRefinementSuccess,
                          "Aggregate Refinement Choice" : aggregateRefinementChoice,
                          "Model Better? (Based on Brier Score - Success)":modelBetterThanAggSuccess,
                          "Model Better? (Based on Brier Score - Choice)":modelBetterThanAggChoice,
                          "Average Success":meanSuccess,
                          "Average Correct Chocie":meanChoice,
                          "Object Permanence Ability Mean (All)": OPAbilityAllMean,
                          "Visual Acuity Ability Mean (All)" : visualAcuityAbilityAllMean,
                          "Flat Navigation Ability Mean (All)" : flatNavAbilityAllMean,
                          "Lava Ability Mean (All)" : lavaAbilityAllMean,
                          "Right Ability Mean (All)" : rightAbilityAllMean,
                          "Left Ability Mean (All)" : leftAbilityAllMean,
                          "Ahead Ability Mean (All)" : aheadAbilityAllMean,
                          "Object Permanence Ability SD (All)": OPAbilityAllSD,
                          "Visual Acuity Ability SD (All)" : visualAcuityAbilityAllSD,
                          "Flat Navigation Ability SD (All)" : flatNavAbilityAllSD,
                          "Lava Ability SD (All)" : lavaAbilityAllSD,
                          "Right Ability SD (All)" : rightAbilityAllSD,
                          "Left Ability SD (All)" : leftAbilityAllSD,
                          "Ahead Ability SD (All)" : aheadAbilityAllSD,
                          "Object Permanence Ability Min (All)": OPAbilityAllMin,
                          "Visual Acuity Ability Min (All)" : visualAcuityAbilityAllMin,
                          "Flat Navigation Ability Min (All)" : flatNavAbilityAllMin,
                          "Lava Ability Min (All)" : lavaAbilityAllMin,
                          "Right Ability Min (All)" : rightAbilityAllMin,
                          "Left Ability Min (All)" : leftAbilityAllMin,
                          "Ahead Ability Min (All)" : aheadAbilityAllMin,
                          "Object Permanence Ability Max (All)": OPAbilityAllMax,
                          "Visual Acuity Ability Max (All)" : visualAcuityAbilityAllMax,
                          "Flat Navigation Ability Max (All)" : flatNavAbilityAllMax,
                          "Lava Ability Max (All)" : lavaAbilityAllMax,
                          "Right Ability Max (All)" : rightAbilityAllMax,
                          "Left Ability Max (All)" : leftAbilityAllMax,
                          "Ahead Ability Max (All)" : aheadAbilityAllMax,})
  finalDF.to_csv("/content/drive/Shareddrives/comparative-object-permanence/measurement_layout_results/AgentResults.csv", index = False)

  ## figures

  for ability in abilitiesToShow:
    trace_plot = az.plot_trace(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/traceplot_{agent}_{ability}.png")

    forest_plot = az.plot_forest(data=data_all['posterior'][[ability]])
    plt.savefig(f"{figures_folder}/forest_{agent}_{ability}.png")

  energy_plot = az.plot_energy(data=data_all)
  plt.savefig(f"{figures_folder}/energyplot_{agent}.png")

  summary = az.summary(data_all['posterior'][abilitiesToShow])
  save_object(summary, f"{full_data_model_folder}/summary_{agent}.pkl")
  summary.to_csv(f"{full_data_model_folder}/summary_{agent}.csv", index = False)

  print(f"Agent: {agent}")
  print(summary)
